## <center> **<span style="color: red">NLP</span> steps and explnation.**

Here we will use <span style="color: #ffc83d">**Hugging face**</span> trasformers pre trained models becuase it'll take so much effort to label<br>
all of our text columns "<ins>Comments, Describtion, Title</ins>" and also we will extract categorical and numirecal<br>
features from the text columns.<br><br>

##### **Steps in detail:**
<ul>
    <li>Extract categorical features from text columns sush as:<ol> 
        <li>Contains emojies
        <li>Positive or negative
        <li>language
        <li>typos</ol>
    <li> Doing analysis for most common words in comments and titles
    <li> Extracing the length of the comments, descriptions & titles
    <li> Saving those new data into new <strong>one</strong> Df to use in the final ML model

## <center> **Importing the <span style="color: red">Packeges</span>**

In [10]:
import nltk
import sqlite3
import transformers
import wordcloud
import pandas as pd
import numpy as np
import emoji

## <center> **Reading the <span style="color: red">data</span>**

In [3]:
con = sqlite3.connect('../database.db')

df = pd.read_sql_query("""   
                        
                        SELECT *
                        FROM base_videos AS bv

                        INNER JOIN base_channels  AS bc ON
                            bc.channel_name = bv.channelTitle

                        INNER JOIN comments  AS c ON
                            c.video_id = bv.video_id""", con)

con.close()

df = df.T.drop_duplicates().T

In [30]:
df.head()

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,commentCount,definition,duration_in_minutes,subscribers,total_views,date,video_count,about,country,comments,like_counts,reply_counts
0,N6dYG0FwFvk,Tribe Gaming,BEST MOMENTS OF TRIBE GAMING! (2022),TRIBE GAMING - BEST OF 2022! \n\n2022 was a he...,"['Tribe', 'Tribe Gaming', 'Gaming', 'Mobile', ...",2023-01-17 19:15:01+00:00,45177,2540,292,hd,14.835938,488000,101639693,2017-02-27 00:00:00,453,Mobile gaming content and esports organization...,US,2022 has been amazing. Can’t wait for 2023 Tri...,3,0
1,N6dYG0FwFvk,Tribe Gaming,BEST MOMENTS OF TRIBE GAMING! (2022),TRIBE GAMING - BEST OF 2022! \n\n2022 was a he...,"['Tribe', 'Tribe Gaming', 'Gaming', 'Mobile', ...",2023-01-17 19:15:01+00:00,45177,2540,292,hd,14.835938,488000,101639693,2017-02-27 00:00:00,453,Mobile gaming content and esports organization...,US,3:20 THAT'S WILD 💀💀💀,0,0
2,N6dYG0FwFvk,Tribe Gaming,BEST MOMENTS OF TRIBE GAMING! (2022),TRIBE GAMING - BEST OF 2022! \n\n2022 was a he...,"['Tribe', 'Tribe Gaming', 'Gaming', 'Mobile', ...",2023-01-17 19:15:01+00:00,45177,2540,292,hd,14.835938,488000,101639693,2017-02-27 00:00:00,453,Mobile gaming content and esports organization...,US,"I know this is a couple of weeks old, but does...",1,0
3,N6dYG0FwFvk,Tribe Gaming,BEST MOMENTS OF TRIBE GAMING! (2022),TRIBE GAMING - BEST OF 2022! \n\n2022 was a he...,"['Tribe', 'Tribe Gaming', 'Gaming', 'Mobile', ...",2023-01-17 19:15:01+00:00,45177,2540,292,hd,14.835938,488000,101639693,2017-02-27 00:00:00,453,Mobile gaming content and esports organization...,US,I miss Pat,0,0
4,N6dYG0FwFvk,Tribe Gaming,BEST MOMENTS OF TRIBE GAMING! (2022),TRIBE GAMING - BEST OF 2022! \n\n2022 was a he...,"['Tribe', 'Tribe Gaming', 'Gaming', 'Mobile', ...",2023-01-17 19:15:01+00:00,45177,2540,292,hd,14.835938,488000,101639693,2017-02-27 00:00:00,453,Mobile gaming content and esports organization...,US,I miss Pat in these videos.,1,0


##  <center> **Extract <span style="color: red">categorical</span> features**

#### *Extract emojies count per text*

In [26]:
comments_emojis_counts: list = []
title_emojis_counts: list = []
desc_emojis_counts: list = []

for comment, title, desc in zip(df["comments"], df["title"], df["description"]):
    
    comments_emojis_count: int = 0
    title_emojis_count: int = 0
    desc_emojis_count: int = 0
    
    for comment_char, title_char, desc_char in zip(comment, title, desc):
        
        if emoji.is_emoji(comment_char):
            comments_emojis_count += 1
            
        if emoji.is_emoji(title_char):
            title_emojis_count += 1
            
        if emoji.is_emoji(desc_char):
            desc_emojis_count += 1
    
    comments_emojis_counts.append(comments_emojis_count)
    title_emojis_counts.append(title_emojis_count)
    desc_emojis_counts.append(desc_emojis_count)

In [30]:
df["comments_emojis_count"] = comments_emojis_counts
df["title_emojis_count"] = title_emojis_counts
df["desc_emojis_count"] = desc_emojis_counts

df.head()

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,commentCount,definition,...,about,country,comments,like_counts,reply_counts,comments_emoies_count,title_emoies_count,comments_emojis_count,title_emojis_count,desc_emojis_count
0,N6dYG0FwFvk,Tribe Gaming,BEST MOMENTS OF TRIBE GAMING! (2022),TRIBE GAMING - BEST OF 2022! \n\n2022 was a he...,"['Tribe', 'Tribe Gaming', 'Gaming', 'Mobile', ...",2023-01-17 19:15:01+00:00,45177,2540,292,hd,...,Mobile gaming content and esports organization...,US,2022 has been amazing. Can’t wait for 2023 Tri...,3,0,0,0,0,0,0
1,N6dYG0FwFvk,Tribe Gaming,BEST MOMENTS OF TRIBE GAMING! (2022),TRIBE GAMING - BEST OF 2022! \n\n2022 was a he...,"['Tribe', 'Tribe Gaming', 'Gaming', 'Mobile', ...",2023-01-17 19:15:01+00:00,45177,2540,292,hd,...,Mobile gaming content and esports organization...,US,3:20 THAT'S WILD 💀💀💀,0,0,3,0,3,0,0
2,N6dYG0FwFvk,Tribe Gaming,BEST MOMENTS OF TRIBE GAMING! (2022),TRIBE GAMING - BEST OF 2022! \n\n2022 was a he...,"['Tribe', 'Tribe Gaming', 'Gaming', 'Mobile', ...",2023-01-17 19:15:01+00:00,45177,2540,292,hd,...,Mobile gaming content and esports organization...,US,"I know this is a couple of weeks old, but does...",1,0,0,0,0,0,0
3,N6dYG0FwFvk,Tribe Gaming,BEST MOMENTS OF TRIBE GAMING! (2022),TRIBE GAMING - BEST OF 2022! \n\n2022 was a he...,"['Tribe', 'Tribe Gaming', 'Gaming', 'Mobile', ...",2023-01-17 19:15:01+00:00,45177,2540,292,hd,...,Mobile gaming content and esports organization...,US,I miss Pat,0,0,0,0,0,0,0
4,N6dYG0FwFvk,Tribe Gaming,BEST MOMENTS OF TRIBE GAMING! (2022),TRIBE GAMING - BEST OF 2022! \n\n2022 was a he...,"['Tribe', 'Tribe Gaming', 'Gaming', 'Mobile', ...",2023-01-17 19:15:01+00:00,45177,2540,292,hd,...,Mobile gaming content and esports organization...,US,I miss Pat in these videos.,1,0,0,0,0,0,0


#### *Language detection & transilation*

In [31]:
language_detector = transformers.pipeline("text2text-generation", model="joeddav/xlm-roberta-large-xnli")

translated_comments = []
translated_descs = []
translated_titels = []

for comment, title, desc in zip(df["comments"], df["title"], df["description"]):
    
    comment_language = language_detector(comment, max_length=512)[0]['generated_text'].strip()
    title_language = language_detector(title, max_length=512)[0]['generated_text'].strip()
    desc_language = language_detector(desc, max_length=512)[0]['generated_text'].strip()
    
    if comment_language == 'en':
        translated_comments.append(comment)
        
    else:
        translator = pipeline(f"translation_{comment_language}_to_en")
        translated_comment = translator(comment, max_length= 128)[0]['translation_text']
        translated_data.append(translated_text)

OSError: joeddav/xlm-roberta-large-xnli is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo with `use_auth_token` or log in with `huggingface-cli login` and pass `use_auth_token=True`.

In [ ]:
translated_comments

#### *Text nature classification*

In [ ]:
language_detector = pipeline("text2text-generation", model="joeddav/xlm-roberta-large-xnli")
